In [1]:
import numpy as np 
import tensorflow as tf
import pickle
from tensorflow.keras import  preprocessing, utils
from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers.recurrent import LSTM,SimpleRNN
import yaml
import os
import pandas as pd
# from numba import jit, cuda
import pickle
from statistics import mode

In [2]:
print(len(tf.config.experimental.list_physical_devices('GPU')))

1


In [3]:
print("Num GPUs Available: ", (tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
files_list =  os.listdir('dataset/Kaggledataset/')
# files_list = ['dataset/Kaggledataset/food.yml', 'dataset/Kaggledataset/greetings.yml','dataset/Kaggledataset/humor.yml','dataset/Kaggledataset/movies.yml']

In [5]:
questions = list()
answers = list()

for filepath in files_list:
    stream = open('dataset/Kaggledataset/' +filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

In [6]:
# questions = list()
# answers = list()
# data_path = 'dataset/human_chat.txt'
# with open(data_path, encoding='utf8') as file:
#     for i , line in enumerate(file):
#         try:
#             if line[6] == '1':
#                 questions.append(line[9:])
#                 # questions1.append(line[8:])
#             else:
#                 answers.append(line[9:])
#                 # answers1.append(line[8:])
#         except:
#             print("error at line :" , i)
        
#         # print(i , len(questions) , len(answers))


In [7]:
data_path = 'dataset/dialogs.txt'
# questions = list()
# answers = list()
with open(data_path, encoding='utf8') as file:
    for i , line in enumerate(file):
        q , a =line.split('\t')
        questions.append(q)
        answers.append(a)

In [8]:
topicat_chat  = pd.read_csv('dataset/topical_chat.csv')
topicat_chat.head()
topicat_chat.sentiment.unique()

array([' Curious to dive deeper', ' Happy', ' Neutral', ' Surprised',
       ' Disgusted', ' Sad', ' Fearful', ' Angry'], dtype=object)

In [9]:
mental_health = pd.read_csv('dataset/Mental_Health_FAQ.csv')
mental_health.head()

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [10]:
data = list(zip(mental_health.Questions, mental_health.Answers))
for i in data:
    questions.append(i[0])
    answers.append(i[1])

In [11]:
del data
del mental_health

In [12]:
jeopardy = pd.read_csv('dataset/JEOPARDY_CSV.csv')
jeopardy.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [13]:
Q , A = jeopardy[' Question'] , jeopardy[' Answer']
questions.extend(Q)
answers.extend(A)

In [14]:
del Q
del A
del jeopardy

In [15]:
emotion_emotion = pd.read_csv('dataset/emotion-emotion_69k.csv')
emotion_emotion.head()
emotion_emotion.emotion.unique()

array(['sentimental', 'afraid', 'proud', 'faithful', 'terrified',
       'joyful', 'angry', 'sad', 'jealous', 'grateful', 'prepared',
       'embarrassed', 'excited', 'annoyed', 'lonely', 'ashamed', 'guilty',
       'surprised', 'nostalgic', 'confident', 'furious', 'disappointed',
       'caring', 'trusting', 'disgusted', 'anticipating', 'anxious',
       'hopeful', 'content', 'impressed', 'apprehensive', 'devastated',
       ' I really killed it!', ' we were in a different country',
       't even like scary things',
       't believe I like the show Power so much. I was never really into shows like that',
       nan, ' time to jump on the motorcycle and go cruising!',
       " a boy.  I hear all these different labor stories that aren't exactly reassuring!  ",
       't believe my daughter taught herself how to play the ukelele. I was amazed',
       't think I wold like super heroes',
       "m so mad with my brother. He stole from me and didn't think I would notice. ",
       " but

In [16]:
for i in range(len(emotion_emotion['empathetic_dialogues'])):
    qn = emotion_emotion['empathetic_dialogues'][i][10:-9]
    if len(qn) != 0 and len(emotion_emotion['labels'][i]) !=0:
        questions.append(qn)
        answers.append(emotion_emotion['labels'][i])

In [17]:
print(len(questions) , len(answers))

285864 285864


In [18]:
line  = list()
emotion = list()

In [19]:
for i in range(len(emotion_emotion['Situation'])):
    situation = emotion_emotion['Situation'][i]
    if situation not in line:
        line.append(str(situation))
        emotion.append(str(emotion_emotion['emotion']))

In [20]:
print(len(line) , len(emotion))

19205 19205


In [21]:
for i in range(len(topicat_chat['message'])):     
    line.append(topicat_chat['message'][i])
    emotion.append(topicat_chat['sentiment'][i])

In [22]:
print(len(line) , len(emotion))

207583 207583


In [23]:
print(type(line[0]))

<class 'str'>


In [24]:
unique_emotion  = set()
for i in emotion:
    unique_emotion.add(i)
len(unique_emotion)

9

In [25]:
print(unique_emotion)

{' Neutral', '0        sentimental\n1        sentimental\n2        sentimental\n3        sentimental\n4        sentimental\n            ...     \n64631    sentimental\n64632    sentimental\n64633      surprised\n64634      surprised\n64635      surprised\nName: emotion, Length: 64636, dtype: object', ' Curious to dive deeper', ' Surprised', ' Happy', ' Sad', ' Angry', ' Disgusted', ' Fearful'}


In [26]:
# import cleantext
# @jit(target_backend='cuda')
# def clean(x,y):
#      Questions = list()
#      Answers = list()
#      QA = list()
#      for i , sentences in enumerate(x):
#           clean_question = (cleantext.clean(str(x[i]), extra_spaces=True, lowercase=True, stopwords=False, stemming=True, numbers=True, punct=True, clean_all = True))
#           clean_answers = (cleantext.clean(str(y[i]), extra_spaces=True, lowercase=True, stopwords=False, stemming=True, numbers=True, punct=True, clean_all = True))
#           qn = (( '<START> ' + clean_question + ' <END>' ).split()) 
#           an = (( '<START> ' + clean_answers + ' <END>' ).split()) 
#           Questions.append(qn)
#           Answers.append(an)
#           QA.append(qn)
#           QA.append(an)
#      return Questions , Answers ,QA
    
    

In [27]:
# Questions , Answers , QA = clean(questions,answers)

In [28]:
# len(Questions) , len(Answers) , len(QA)

In [29]:
# with open('QA.pickle','wb') as f:
#     pickle.dump([Questions,Answers,QA],f)


In [30]:
# with open('QA.pickle','rb') as f:
#     Questions,Answers,QA =pickle.load(f)  

In [31]:
# from gensim.models import Word2Vec

In [32]:
# word2vec = Word2Vec(QA , min_count=0 , window=5,workers=6)

In [33]:
# print(word2vec.wv['what'].shape)

In [34]:
# @jit(target_backend='cuda')  
# def replacer(model,A):
#     X = list()
#     for  i , sentences in enumerate(A):
#         # print(type(len(sentences)))
#         q = np.zeros((len(sentences),100),float)
#         # try:
#         for j ,words in enumerate(sentences):
#                 q[j] =(model.wv[words].flatten())
#         X.append(q)
#         # except:
#         #     print("Error in  line no :" , i ,"\n")
#     return X


In [35]:
# train_input = replacer(word2vec,Questions)

In [36]:
# train_output = replacer(word2vec,Answers)

In [37]:
# train_input = np.array(train_input)
# train_output = np.array(train_output)

In [38]:
answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

In [39]:
# import re
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 108717


In [40]:
vocab = []
for word in tokenizer.word_index:
  vocab.append(word)

In [41]:
maxlen_questions = 10
maxlen_answers = 10

In [42]:
tokenized_questions = tokenizer.texts_to_sequences( questions  )
# maxlen_questions = mode( [len(i) for i in tokenized_questions ] )
# print(maxlen_questions)
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post' ,truncating= 'post')
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape, maxlen_questions)

(285860, 10) 10


In [43]:
tokenized_answers = tokenizer.texts_to_sequences( answers  )
# maxlen_answers = mode( [len(i) for i in tokenized_answers ] )

padded_answers = np.array(preprocessing.sequence.pad_sequences( tokenized_answers , maxlen= maxlen_answers , padding='post',truncating= 'post' ))
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

(285860, 10) 10


In [44]:
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers1 = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post',truncating= 'post'  )
# onehot_answers = utils.to_categorical( padded_answers1 , num_classes= VOCAB_SIZE ,dtype = 'float32' )
decoder_output_data = np.array( padded_answers1 )
print( decoder_output_data.shape )

(285860, 10)


In [45]:
# tokenized_answers = tokenizer.texts_to_sequences( answers[:5000]   )
# tokenized_questions = tokenizer.texts_to_sequences( questions[:5000]   )
# maxlen_questions = mode( [len(i) for i in tokenized_questions ] )
# maxlen_answers = mode( [len(i) for i in tokenized_answers ] )

In [46]:
# def my_numpy_func(y_true, y_pred):
#   # y_true and y_pred are already numpy arrays
#   # put all numpy or sklearn codes here
#   y_true = utils.to_categorical( y_true , num_classes= VOCAB_SIZE ,dtype = 'float32' )
#   cce = tf.keras.losses.CategoricalCrossentropy()
#   return cce(y_true, y_pred)

# @tf.function
# def custom_metric(y_true, y_pred):
#   # y_true and y_pred are tensors
#   # no numpy or sklearn code is allowed here
#   score = tf.numpy_function(my_numpy_func, [y_true, y_pred], tf.float32)
#   return score

In [71]:
def custom_loss_function(y_true, y_pred):
   y1 = tf.numpy_function(utils.to_categorical , [y_true ,VOCAB_SIZE] , tf.float32) 
   cce = tf.keras.losses.CategoricalCrossentropy()
   # y2 = utils.to_categorical(y_true , VOCAB_SIZE )
   # print(y1[0] , y2[0])
   loss = cce(y1, y_pred)
   # del y1
   return loss

In [85]:
# encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
# encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
# encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
# encoder_states = [ state_h , state_c ]

# decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
# decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
# decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
# decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
# decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
# output = decoder_dense ( decoder_outputs )

# model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )


In [93]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 100 , mask_zero=True ) (encoder_inputs)
# encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 100 , return_state=True )( encoder_embedding )
# encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 100 , mask_zero=True) (decoder_inputs)

decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
concatenate = tf.keras.layers.Concatenate( name='Concatenate_1')([encoder_embedding,decoder_embedding])
decoder_outputs , _ , _ = decoder_lstm ( concatenate )

decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )

In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 100 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 100 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 100 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 100 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_lstm1 = tf.keras.layers.LSTM( 100 , return_state=True , return_sequences=True )
decoder_outputs1 , _ , _ = decoder_lstm1 ( decoder_outputs )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs1 )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )

In [94]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss=custom_loss_function , run_eagerly= False )
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 10)]         0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 10)]         0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 10, 100)      10871700    input_13[0][0]                   
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 10, 100)      10871700    input_14[0][0]                   
____________________________________________________________________________________________

In [95]:
model.fit([encoder_input_data, decoder_input_data], decoder_output_data, batch_size= 50, epochs= 3 ) 
model.save( 'mo.h5' )

Epoch 1/3


In [ ]:
# class DataGenerator(tf.keras.utils.Sequence):
#   def __init__(self, x, x1, y, batch_size = 38, shuffle=True):
#     super().__init__()
#     self.x = x
#     self.x1 = x1
#     self.y = y
#     self.batch_size = batch_size
#     self.shuffle = shuffle
#     key_array = []
#     self.key_array = np.arange(self.x.shape[0])
#     self.on_epoch_end()

#   def __len__(self):
#     return len(self.key_array)//self.batch_size

#   def __getitem__(self, index):
#     keys = self.key_array[index*self.batch_size:(index+1)*self.batch_size]
#     x = np.asarray(self.x[keys], dtype=np.float32)
#     x1 = np.asarray(self.x1[keys], dtype=np.float32)
#     y = np.asarray(self.y[keys], dtype=np.float32)
#     return x,x1, y

#   def on_epoch_end(self):
#     if self.shuffle:
#       self.key_array = np.random.permutation(self.key_array)
      

# generator = DataGenerator(x=encoder_input_data, x1 =decoder_input_data , y = decoder_output_data, batch_size= 50, shuffle=True)

In [ ]:
# len(generator)
# print( encoder_input_data.shape )
# print( decoder_input_data.shape )
# print( decoder_output_data.shape )

In [ ]:
# from sklearn.metrics import accuracy_score

In [ ]:
# import time

In [ ]:
# len(generator)/

In [ ]:
# n_epochs = 10
# n_batches = len(generator)
# loss_train = np.zeros(shape=(n_epochs,), dtype=np.float32)
# acc_train = np.zeros(shape=(n_epochs,), dtype=np.float32)
# loss_val = np.zeros(shape=(n_epochs,))
# acc_val = np.zeros(shape=(n_epochs,))
# opt = tf.keras.optimizers.RMSprop()
# cce = tf.keras.losses.CategoricalCrossentropy()
# for epoch in range(n_epochs):
#   print("\nEpoch = %d" % (epoch,))
#   epoch_loss_avg = tf.keras.metrics.Mean() # Keeping track of the training loss
#   epoch_acc_avg = tf.keras.metrics.Mean() # Keeping track of the training accuracy
#   start = time.time()
#   for batch in range(n_batches):
#     x1 ,x2, y = generator[batch]
#     # print(len(y))
#     y1 = utils.to_categorical( y , num_classes= VOCAB_SIZE ,dtype = 'float32' )

#     with tf.GradientTape() as tape: # Forward pass
#       y_ = model((x1 , x2), training=True)
#       loss = cce(y1 , y_)
      
#     grad = tape.gradient(loss, model.trainable_variables) # Backpropagation
#     # print(grad)
#     opt.apply_gradients(zip(grad, model.trainable_variables)) # Update network weights
    
#     epoch_loss_avg(loss)
#     # epoch_acc_avg(accuracy_score(y_true=y, y_pred=np.argmax(y_, axis=-1)))
#     if batch % 600 == 0:
#             print(
#                 "Epoch %d : Batch %d : Loss %.4f"
#                 % (epoch , batch, float(loss))
#             )

  
#   end = time.time()
#   loss_train[epoch] = epoch_loss_avg.result()
#   acc_train[epoch] = epoch_acc_avg.result()

#   print('---- Training ----')
#   print('Epoch =  {0:.3f}'.format(epoch))
#   print('Loss  =  {0:.3f}'.format(loss_train[epoch]))
#   print('Acc   =  {0:.3f}'.format(acc_train[epoch]))
#   print('Time  =  {0:.3f}'.format(end - start))

In [ ]:
def custom_loss_function(y_true, y_pred):
   cce = tf.keras.losses.CategoricalCrossentropy()
   y_true = utils.to_categorical( y_true , num_classes= VOCAB_SIZE ,dtype = 'float32' )
   loss = cce(y_true, y_pred).numpy()
   return tf.add(loss , 0.0)

In [ ]:
model.save('custom1.h5')

In [ ]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [ ]:
def str_to_tokens( sentence : str ):

    words = sentence.lower().split()
    tokens_list = list()
  
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [ ]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )